# MNIST Baseline

Just trying to repro the results of the first half of Chapter 4 in Fast.ai book. 

We're pulling the MNIST data set and creating simple baseline for 3-vs-7 discrimination.

Note: I'm doing this with as little referencing the book as possible.

## Pulling down the data and taking a look

In [1]:
from fastai.data.external import untar_data

In [2]:
from fastai.data.external import URLs

In [3]:
mnist = untar_data(URLs.MNIST_SAMPLE)

In [4]:
threes_train = mnist/'train'/'3'
sevens_train = mnist/'train'/'7'

In [5]:
threes_train = threes_train.ls()
sevens_train = sevens_train.ls()

In [6]:
from PIL import *

In [7]:
image = Image.open(threes_train[0])
image

## Getting Sample Image Into Into A Tensor

Note: I'm not totally sure what the fastbook import below is giving me. I tried to do things without it, but the image didn't seem to actually be encoded as a series of uint8's. I'm not totally sure how that transformation is taking place. Maybe I'll investigate later on...

In [9]:
from fastbook import *

three_tens = tensor(image, dtype=torch.uint8)

In [10]:
three_tens.shape

torch.Size([28, 28])

In [11]:
import pandas as pd

In [12]:
df = pd.DataFrame(three_tens)
df.style.set_properties(**{'font-size':'6pt'}).background_gradient('Greys')

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,211,254,187,166,166,139,79,79,63,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,122,214,253,253,253,254,253,253,251,199,145,63,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,8,17,51,134,105,128,191,191,202,253,251,129,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,57,253,251,159,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,96,253,253,244,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,96,245,253,244,96,0,0,0,0,0,0,0


### Creating a Mean Out of the 3 and 7 Validation Images

In [14]:
three_tens = [tensor(Image.open(item), dtype = torch.uint8) for item in threes_train]
seven_tens = [tensor(Image.open(item), dtype = torch.uint8) for item in sevens_train]

In [29]:
# NOTE: I had previously _not_ normalized here and did it below as an experiment.
#       I'm putting it here so the entire notebook reads better.
threes_stacked = torch.stack(three_tens).float()/255
sevens_stacked = torch.stack(seven_tens).float()/255

In [30]:
threes_stacked.shape

torch.Size([6131, 28, 28])

In [31]:
three_mean = threes_stacked.mean(0)
three_mean.shape

torch.Size([28, 28])

In [32]:
seven_mean = sevens_stacked.mean(0)
seven_mean.shape

torch.Size([28, 28])

## Comparing a Single Validation Example to a Mean Element-Wise

In [33]:
valid_3 = Image.open((mnist/'valid'/'3').ls()[0])
valid_3

In [34]:
valid_3_tens = tensor(valid_3, dtype=torch.uint8)/255

In [35]:
diff = three_mean - valid_3_tens
diff[4:10, 4:10]

tensor([[ 0.0014,  0.0058,  0.0137,  0.0301,  0.0554,  0.0903],
        [-0.0342,  0.0278,  0.0611,  0.1127,  0.1869,  0.2489],
        [-0.7755, -0.6656, -0.6051, -0.2003, -0.1826, -0.3410],
        [-0.9582, -0.9132, -0.8423, -0.7545, -0.6584, -0.5693],
        [-0.7199, -0.9173, -0.8617, -0.6760, -0.5767, -0.5326],
        [-0.0491, -0.2078, -0.3380,  0.0529,  0.1417,  0.1558]])

## Turning that Mean into a Loss Measure (both L1 and L2 / Abs Mean Error and Squared Mean Error)

In [70]:
def L1_measure(a, b):
    return (a-b).abs().mean((-1, -2))

def L2_measure(a, b):
    return (a-b).square().mean((-1, -2))

In [71]:
print(L1_measure(three_mean, valid_3_tens))
print(L2_measure(three_mean, valid_3_tens))

tensor(0.1488)
tensor(0.0717)


## Getting Loss and Accuracy Measures for Entirety of Validation Set

In [82]:
valid_3s = [tensor(Image.open(item), dtype=torch.uint8) for item in (mnist/'valid'/'3').ls()]
valid_7s = [tensor(Image.open(item), dtype=torch.uint8) for item in (mnist/'valid'/'7').ls()]
print(len(valid_3s))
print(len(valid_7s))

1010
1028


In [83]:
valid_3s_stacked = torch.stack(valid_3s)/255
print(valid_3s_stacked.shape) 

valid_7s_stacked = torch.stack(valid_7s)/255
print(valid_7s_stacked.shape)

torch.Size([1010, 28, 28])
torch.Size([1028, 28, 28])


In [89]:
three_check = L1_measure(three_mean, valid_3s_stacked)
seven_check = L1_measure(three_mean, valid_7s_stacked)

print(three_check.shape)
print(seven_check.shape)
print(three_check)
print(seven_check)
print(three_check.mean())
print(seven_check.mean())

torch.Size([1010])
torch.Size([1028])
tensor([0.1488, 0.1145, 0.1158,  ..., 0.1129, 0.1419, 0.1669])
tensor([0.1560, 0.1482, 0.1751,  ..., 0.1293, 0.1346, 0.1487])
tensor(0.1296)
tensor(0.1590)


In [91]:
def is_3(a):
    return L1_measure(a, three_mean) < L1_measure(a, seven_mean)

In [93]:
is_3(valid_3_tens)

tensor(True)

In [101]:
three_accuracy = is_3(valid_3s_stacked).float().mean()
three_accuracy

tensor(0.9168)

In [103]:
seven_accuracy = (1 - is_3(valid_7s_stacked).float()).mean()
seven_accuracy

tensor(0.9854)